In [1]:
# Advent of Code 2022
# Day 15
# https://adventofcode.com/2022/day/15

import re

with open('input.txt') as f:
    data = f.read()
    lines = data.split('\n')
    lines = list(filter(None, lines))



In [2]:
# parse strings to get sensor and beacon coordinates
def get_coords(line):
    re_sensors_beacons = re.compile(r'(\-*\d+)')
    xy_nums = list(re_sensors_beacons.findall(line))
    for i, num in enumerate(xy_nums):
        xy_nums[i] = int(num)
    return (xy_nums)



In [3]:
# # set of all beacons
# beacon_points = set()
# for l in lines:
#     coords = get_coords(l)
#     x = coords[2]
#     y = coords[3]
#     coords = (x, y)
#     beacon_points.add(coords)

# set of all beacons in a given row
def get_beacon_points_row(test_row):
    beacon_points_row = set()
    for l in lines:
        coords = get_coords(l)
        x = coords[2]
        y = coords[3]
        if y == test_row:
            coords = (x, y)
            beacon_points_row.add(coords)
    return beacon_points_row


# # set of all sensors
# sensor_points = set()
# for l in lines:
#     coords = get_coords(l)
#     x = coords[0]
#     y = coords[1]
#     coords = (x, y)
#     sensor_points.add(coords)

# set of all beacons in a given row
def get_sensor_points_row(test_row):
    sensor_points_row = set()
    for l in lines:
        coords = get_coords(l)
        x = coords[0]
        y = coords[1]
        if y == test_row:
            coords = (x, y)
            sensor_points_row.add(coords)
    return sensor_points_row


get_beacon_points_row(3)


set()

In [4]:
def get_dist (line):
    sx, sy, bx, by = get_coords(line)
    dist = abs(sx-bx) + abs(sy-by)
    return dist

In [5]:
# list all points within range of a sensor-beacon pair
# but only if they are also within range of a given row
def get_nearby_points(line, test_row):

    # generate a set to contain all beaconless points
    nearby = set()

    # identify sensor and beacon x-y coordinates
    sx, sy, bx, by = get_coords(line)


    # get distance between beacon and sensor
    dist = get_dist(line)

    # test to see if the area covered by this
    # sensor-beacon pair is within range of the
    # test line
    if abs(sy-test_row) <= dist:

        # # for each x value, caluclate min and max y values
        # for x in range(sx-dist, sx+dist+1):
        #     x_dist = abs(x-sx)
        #     y_dist = dist - x_dist
        #     y_min = sy - y_dist
        #     y_max = sy + y_dist

        #     # add all points between min and max x-y coords to the set
        #     points_list = [(x,y) for y in range(y_min, y_max+1)]
        #     nearby.update(points_list)

        y_dist = abs(sy - test_row)
        x_dist = dist - y_dist
        x_min = sx - x_dist
        x_max = sx + x_dist
        points_list = [(x, test_row) for x in range(x_min, x_max+1)]
        nearby.update(points_list)
    
    return nearby




In [6]:
# get all of the points within range of all beacon-sensor pairs near
# the test row

def get_all_points_near(test_row):
    
    all_nearby_points = set()
    for l in lines:
        all_nearby_points.update(get_nearby_points(l, test_row))

    # eliminate points where beacons or sensors are already known to exist:
    beaconless = all_nearby_points - get_beacon_points_row(test_row) - get_sensor_points_row(test_row)

    return beaconless



In [7]:
# Count beaconless points for a given row:
def count_nearby(test_row):

    beaconless = get_all_points_near(test_row)

    nearby_row_list = [i for i in beaconless if i[1]==test_row]
    return len(nearby_row_list)

answer = count_nearby(2000000)

print(answer)



4560025


In [8]:
# Par 2:  With x and y both in the range 0 to <some max number>, find the only
# point where a beacon could be.  Tuning frequency = x * 4000000 + y.



# revised version includes all sensor and beacon points
def get_all_points_near_incl_sb(test_row, max_index):
    
    all_nearby_points = set()
    for l in lines:
        nearby_points = get_nearby_points(l, test_row)
        # filter to remove points outside the max index range
        nearby_points = {n for n in nearby_points if 0 <= n[1] <= max_index}
        all_nearby_points.update(nearby_points)

    # # eliminate points where beacons or sensors are already known to exist:
    # beaconless = all_nearby_points - get_beacon_points_row(test_row) - get_sensor_points_row(test_row)

    return all_nearby_points


# Exclude all points that are either already occupied OR known to be empty
# (anything within a sensor-beacon range).  Only consider the ones that fall
# within the specified maximum index for the X or Y value.
def count_excluded_points_row(test_row, max_index):
    
    excluded_points = set()
    for l in lines:
        # all_nearby = list(get_nearby_points(l, test_row))
        excluded_points.update(get_all_points_near_incl_sb(test_row, max_index))

    # narrow excluded points to only the ones in the max_index range
    excluded_points = {p for p in excluded_points if 0 <= p[0] <= max_index}

    # check whether every X value is accounted for in the excluded points.
    all_x_values = {x for x in range(max_index+1)}
    excluded_x_values = {p[0] for p in excluded_points}
        
    # If there is an x value that hasn't been excluded, 
    # that is a possible beacon location. Return it.
    difference = all_x_values - excluded_x_values

    if len(difference) > 0:
        x_excluded = list(difference)[0]
        return x_excluded






In [9]:
# test each row and find the x-y coordinate of the potential beacon spot.
# Return the tuning frequency of that point.

def find_beacon_tf(max_index):
    for y in range(max_index+1):
        x = count_excluded_points_row(y, max_index)
        if x != None:
            
            # calculate tuning frequency
            tf = x * 4000000 + y

            return tf

find_beacon_tf(20)